In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


In [ ]:
## Load the IMDB dataset
max_features = 10000 # voc size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [ ]:
print(f'X_train shape: {X_train.shape}, Training Labels shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape},Testing Labels shape: {y_test.shape}')

In [ ]:
## Inspect a sample review and its label
sample_review = X_train[0]
sample_label = y_train[0]

print(f'Sample Review: {sample_review}')
print(f'Sample Label: {sample_label}')

In [ ]:
#Mapping of words index back to words(for understanding)
word_index = imdb.get_word_index()
# reverse_word_index =
word_index
reverse_word_index = {value: key for key, value in word_index.items()}
reverse_word_index

In [ ]:
decoded_review = ' '.join([reverse_word_index.get(i-3,'?') for i in sample_review])
decoded_review

In [ ]:
from tensorflow.keras.preprocessing import sequence
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
X_train

In [ ]:
X_train[0]

In [ ]:
## Train Simple RNN
model = Sequential()
model.add(Embedding(max_features,128,input_length=max_len)) ## Embedding layers
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
## Train the model with EarlyStopping
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2,callbacks=[earlystopping])

In [ ]:
## Save the model
model.save('simple_rnn_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('simple_rnn_model.h5')

In [ ]:
model.summary()

In [ ]:
## helper fns
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encoded_review])


def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word,2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [ ]:
## Prediction Function
def predict_sentiment(review):
    processed_input = preprocess_text(review)
    prediction = model.predict(processed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [ ]:
## user input and Predicton
example_review = "The movie was fantastic and the acting was great and the plot was thrilling"
sentiment, score = predict_sentiment(example_review)

In [ ]:
print(f'Review {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Score : {score }')